# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import regex as re
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('DisasterData',engine)
X = df['message']
Y = df.iloc[:,4:]
category_names = Y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    Text preprocessing
    
    Arguments:
        text(str): raw text
        
    Outputs:
        clean_tokens(list): tokenized processed text
    """
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower()) # remove punctuation, lowercase
    words = word_tokenize(text) # tokenize
    words = [ w for w in words if w not in stopwords.words("english") ] # remove stopwords
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [ lemmatizer.lemmatize(w).strip() for w in words ] # lemmatize words, remove extra spaces
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(
            tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(
            RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [6]:
import time
start_time = time.time()
pipeline.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 518.9610726833344 seconds ---


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
def metrics(model,X_test,y_test,category_names):
    """
    Evaluate model predictions
    
    Arguments:
        y_test: true classes
        preds: predicted classes
        
    Outputs
        classification_report: classification report for every label
        f1: average F1 score across labels
        accuracy: average accuracy across labels
    """
    preds = model.predict(X_test)
    for i,col in enumerate(category_names):
        print(col)

        f1 = []
        accuracy = []
        y_true = list(y_test.values[:, i])
        y_pred = list(preds[:, i])
        f1.append(f1_score(y_true,y_pred))
        accuracy.append(accuracy_score(y_true,y_pred))
        print(classification_report(y_true, y_pred))

    print("Average F1 score:", np.mean(f1), "\nAverage Accuracy:", np.mean(accuracy))

In [8]:
metrics(pipeline,X_test,y_test,category_names)

related
              precision    recall  f1-score   support

           0       0.75      0.41      0.53      1544
           1       0.84      0.96      0.89      4963

    accuracy                           0.83      6507
   macro avg       0.79      0.68      0.71      6507
weighted avg       0.82      0.83      0.81      6507

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5401
           1       0.82      0.48      0.60      1106

    accuracy                           0.89      6507
   macro avg       0.86      0.73      0.77      6507
weighted avg       0.89      0.89      0.88      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6476
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6304
           1       0.71      0.06      0.11       203

    accuracy                           0.97      6507
   macro avg       0.84      0.53      0.55      6507
weighted avg       0.96      0.97      0.96      6507

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6507

    accuracy                           1.00      6507
   macro avg       1.00      1.00      1.00      6507
weighted avg       1.00      1.00      1.00      6507

water
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6090
           1       0.86      0.39      0.54       417

    accuracy                           0.96      6507
   macro avg       0.91      0.69      0.76      6507
weighted avg       0.95      0.96      0.95      6507

food
              precision    recall  f1-score   supp

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      6507

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6414
           1       0.00      0.00      0.00        93

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.97      0.99      0.98      6507

other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6212
           1       0.17      0.00      0.01       295

    accuracy                           0.95      6507
   macro avg       0.56      0.50      0.49      6507
weighted avg       0.92      0.95      0.

#### PRELIMINARY CONCLUSION

The Random Forest with TfIdfVectorizer and default parameters has the following performances: 

    Average F1 score: 0.4900881057268722 
    
    Average Accuracy: 0.8576917166128785

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fc498883320>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fc498883320>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

#### Parameters for Vectorizer

In [10]:
def build_model1(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(
            tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(
            RandomForestClassifier())),
    ])
    
    parameters = {
        'vect__max_df': [0.8, 1],
        'vect__min_df': [1, 3, 5],
        #'clf__estimator__max_depth': [10, 20],
        #'clf__estimator__min_samples_leaf': [1, 2],
        #'clf__estimator__min_samples_split': [2, 5],
        #'clf__estimator__n_estimators': [100, 200]   
    }
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3)
    
    return cv

In [11]:
cv1 = build_model1()
cv1.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2] END ..vect__max_df=0.8, vect__min_df=1;, score=0.257 total time= 3.9min
[CV 2/2] END ..vect__max_df=0.8, vect__min_df=1;, score=0.246 total time= 3.9min
[CV 1/2] END ..vect__max_df=0.8, vect__min_df=3;, score=0.264 total time= 3.2min
[CV 2/2] END ..vect__max_df=0.8, vect__min_df=3;, score=0.255 total time= 3.3min
[CV 1/2] END ..vect__max_df=0.8, vect__min_df=5;, score=0.265 total time= 3.2min
[CV 2/2] END ..vect__max_df=0.8, vect__min_df=5;, score=0.264 total time= 3.1min
[CV 1/2] END ....vect__max_df=1, vect__min_df=1;, score=0.178 total time=12.3min
[CV 2/2] END ....vect__max_df=1, vect__min_df=1;, score=0.177 total time=11.8min
[CV 1/2] END ......vect__max_df=1, vect__min_df=3;, score=nan total time=  31.6s
[CV 2/2] END ......vect__max_df=1, vect__min_df=3;, score=nan total time=  32.1s
[CV 1/2] END ......vect__max_df=1, vect__min_df=5;, score=nan total time=  31.9s
[CV 2/2] END ......vect__max_df=1, vect__min_df=5

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
4 fits failed out of a total of 12.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_params_steps[name],
  File "/home/natash

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fc498883320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'vect__max_df': [0.8, 1], 'vect__min_df': [1, 3, 5]},
             verbose=3)

In [12]:
cv1.best_params_

{'vect__max_df': 0.8, 'vect__min_df': 5}

#### Parameters for RandomForest 1

In [15]:
def build_model2(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(
            max_df=0.8,
            min_df=5,
            tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(
            RandomForestClassifier())),
    ])
    
    parameters = {
        #'vect__max_df': [0.8, 1],
        #'vect__min_df': [1, 3, 5],
        'clf__estimator__max_depth': [10, 20, None],
        #'clf__estimator__min_samples_leaf': [1, 2],
        #'clf__estimator__min_samples_split': [2, 5],
        'clf__estimator__n_estimators': [100, 200]   
    }
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3)
    
    return cv

In [16]:
cv2 = build_model2()
cv2.fit(X_train, y_train)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2] END clf__estimator__max_depth=10, clf__estimator__n_estimators=100;, score=0.192 total time= 1.4min
[CV 2/2] END clf__estimator__max_depth=10, clf__estimator__n_estimators=100;, score=0.191 total time= 1.4min
[CV 1/2] END clf__estimator__max_depth=10, clf__estimator__n_estimators=200;, score=0.192 total time= 1.7min
[CV 2/2] END clf__estimator__max_depth=10, clf__estimator__n_estimators=200;, score=0.191 total time= 1.8min
[CV 1/2] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100;, score=0.191 total time= 1.6min
[CV 2/2] END clf__estimator__max_depth=20, clf__estimator__n_estimators=100;, score=0.186 total time= 1.6min
[CV 1/2] END clf__estimator__max_depth=20, clf__estimator__n_estimators=200;, score=0.188 total time= 2.0min
[CV 2/2] END clf__estimator__max_depth=20, clf__estimator__n_estimators=200;, score=0.184 total time= 2.3min
[CV 1/2] END clf__estimator__max_depth=None, clf__estimator__n_estim

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(max_df=0.8, min_df=5,
                                                        tokenizer=<function tokenize at 0x7fc498883320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_depth': [10, 20, None],
                         'clf__estimator__n_estimators': [100, 200]},
             verbose=3)

In [17]:
cv2.best_params_

{'clf__estimator__max_depth': None, 'clf__estimator__n_estimators': 200}

#### Parameters for RandomForest 2

In [19]:
def build_model3(): 
    pipeline = Pipeline([
        ('vect', CountVectorizer(
            max_df=0.8,
            min_df=5,
            tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(
            RandomForestClassifier(
                n_estimators=200))),
    ])
    
    parameters = {
        'clf__estimator__min_samples_leaf': [1, 2],
        'clf__estimator__min_samples_split': [2, 5]  
    }
    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3)
    
    return cv

In [20]:
cv3 = build_model3()
cv3.fit(X_train, y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV 1/2] END clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2;, score=0.267 total time= 5.8min
[CV 2/2] END clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=2;, score=0.261 total time= 5.7min
[CV 1/2] END clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5;, score=0.264 total time= 4.9min
[CV 2/2] END clf__estimator__min_samples_leaf=1, clf__estimator__min_samples_split=5;, score=0.258 total time= 5.4min
[CV 1/2] END clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2;, score=0.257 total time= 4.7min
[CV 2/2] END clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=2;, score=0.249 total time= 4.3min
[CV 1/2] END clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=5;, score=0.252 total time= 4.3min
[CV 2/2] END clf__estimator__min_samples_leaf=2, clf__estimator__min_samples_split=5;, score=0.255 total time= 4.3m

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(max_df=0.8, min_df=5,
                                                        tokenizer=<function tokenize at 0x7fc498883320>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=200)))]),
             param_grid={'clf__estimator__min_samples_leaf': [1, 2],
                         'clf__estimator__min_samples_split': [2, 5]},
             verbose=3)

In [21]:
cv3.best_params_

{'clf__estimator__min_samples_leaf': 1, 'clf__estimator__min_samples_split': 2}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
metrics(cv1,X_test,y_test,category_names)

related
              precision    recall  f1-score   support

           0       0.72      0.44      0.55      1544
           1       0.84      0.95      0.89      4963

    accuracy                           0.83      6507
   macro avg       0.78      0.69      0.72      6507
weighted avg       0.82      0.83      0.81      6507

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5401
           1       0.82      0.49      0.62      1106

    accuracy                           0.90      6507
   macro avg       0.86      0.73      0.78      6507
weighted avg       0.89      0.90      0.88      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6476
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6193
           1       0.73      0.18      0.28       314

    accuracy                           0.96      6507
   macro avg       0.85      0.59      0.63      6507
weighted avg       0.95      0.96      0.94      6507

search_and_rescue
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6343
           1       0.50      0.07      0.12       164

    accuracy                           0.97      6507
   macro avg       0.74      0.53      0.55      6507
weighted avg       0.96      0.97      0.97      6507

security
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6387
           1       0.17      0.01      0.02       120

    accuracy                           0.98      6507
   macro avg       0.57      0.50      0.50      6507
weighted avg       0.97      0.98      0.97    

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5957
           1       0.79      0.48      0.60       550

    accuracy                           0.95      6507
   macro avg       0.87      0.74      0.79      6507
weighted avg       0.94      0.95      0.94      6507

clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6419
           1       0.86      0.22      0.35        88

    accuracy                           0.99      6507
   macro avg       0.93      0.61      0.67      6507
weighted avg       0.99      0.99      0.99      6507

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6370
           1       0.62      0.06      0.11       137

    accuracy                           0.98      6507
   macro avg       0.80      0.53      0.55      6507
weighted avg       0.97      0.98      0.97      6507

miss

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6414
           1       0.00      0.00      0.00        93

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.97      0.99      0.98      6507

other_infrastructure
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6212
           1       0.11      0.00      0.01       295

    accuracy                           0.95      6507
   macro avg       0.53      0.50      0.49      6507
weighted avg       0.92      0.95      0.93      6507

weather_related
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      4653
           1       0.83      0.76      0.79      1854

    accuracy                           0.89      6507
   macro avg       0.87      0.85      0.86      6507
weighted avg       0.88      0.89    

In [24]:
metrics(cv2,X_test,y_test,category_names)

related
              precision    recall  f1-score   support

           0       0.72      0.44      0.55      1544
           1       0.84      0.95      0.89      4963

    accuracy                           0.83      6507
   macro avg       0.78      0.69      0.72      6507
weighted avg       0.82      0.83      0.81      6507

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5401
           1       0.81      0.49      0.61      1106

    accuracy                           0.89      6507
   macro avg       0.86      0.73      0.78      6507
weighted avg       0.89      0.89      0.88      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6476
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6387
           1       0.20      0.01      0.02       120

    accuracy                           0.98      6507
   macro avg       0.59      0.50      0.50      6507
weighted avg       0.97      0.98      0.97      6507

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6304
           1       0.65      0.11      0.19       203

    accuracy                           0.97      6507
   macro avg       0.81      0.55      0.59      6507
weighted avg       0.96      0.97      0.96      6507

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6507

    accuracy                           1.00      6507
   macro avg       1.00      1.00      1.00      6507
weighted avg       1.00      1.00      1.00      6507

water
              precision    recall  f1-score   

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6466
           1       0.00      0.00      0.00        41

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6447
           1       0.00      0.00      0.00        60

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.98      0.99      0.99      6507

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      6507

aid

In [25]:
metrics(cv3,X_test,y_test,category_names)

related
              precision    recall  f1-score   support

           0       0.73      0.44      0.55      1544
           1       0.85      0.95      0.89      4963

    accuracy                           0.83      6507
   macro avg       0.79      0.70      0.72      6507
weighted avg       0.82      0.83      0.81      6507

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5401
           1       0.81      0.49      0.61      1106

    accuracy                           0.89      6507
   macro avg       0.86      0.73      0.77      6507
weighted avg       0.89      0.89      0.88      6507

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6476
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6387
           1       0.17      0.01      0.02       120

    accuracy                           0.98      6507
   macro avg       0.57      0.50      0.50      6507
weighted avg       0.97      0.98      0.97      6507

military
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6304
           1       0.65      0.10      0.17       203

    accuracy                           0.97      6507
   macro avg       0.81      0.55      0.58      6507
weighted avg       0.96      0.97      0.96      6507

child_alone
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6507

    accuracy                           1.00      6507
   macro avg       1.00      1.00      1.00      6507
weighted avg       1.00      1.00      1.00      6507

water
              precision    recall  f1-score   

/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/natasha/anaconda3/lib/

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6447
           1       0.00      0.00      0.00        60

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.98      0.99      0.99      6507

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6480
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      6507

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6414
           1       0.00      0.00      0.00        93

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.97      0.99      0.98      6507

o

#### PRELIMINARY CONCLUSION

We trained 4 models (*Random forest model with TfIdf*) and got the folloging F1 and accuracy scores:

1. `pipeline`: Model with default parameters:

    Average F1 score: 0.4900881057268722 
    
    Average Accuracy: 0.8576917166128785


2. `c1`: Model with fine-tuned Vectorizer

    Average F1 score: 0.4819672131147541 
    
    Average Accuracy: 0.8543107422775472
    

3. `c2`: Model with fine-tuned Vectorizer and fine-tuned Random Forest, 1st tuning

    Average F1 score: 0.4760348583877996
    
    Average Accuracy: 0.8521592131550638
    

4. `c3`: Model with fine-tuned Vectorizer and fine-tuned Random Forest, 2nd tuning
    
    Average F1 score: 0.4832069339111592 
    
    Average Accuracy: 0.8533886583679114

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(pipline, open("model.pkl", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.